<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/hp_tuning/01_introduction_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaptado de [Introdução ao sintonizador Keras ](https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=pt-br) dos tutoriais do [Tensorflow Core](https://www.tensorflow.org/tutorials?hl=pt-br).

# Introdução ao sintonizador Keras 

## Visão geral

O Keras Tuner é uma biblioteca que ajuda você a escolher o conjunto ideal de hiperparâmetros para o seu programa TensorFlow. O processo de seleção do conjunto correto de hiperparâmetros para seu aplicativo de aprendizado de máquina (ML) é chamado de *ajuste de hiperparâmetros*(_hyperpameter tuning_) .

Os hiperparâmetros são as variáveis que governam o processo de treinamento e a topologia de um modelo de ML. Essas variáveis permanecem constantes durante o processo de treinamento e afetam diretamente o desempenho do seu programa de ML.

Os hiperparâmetros são de dois tipos:
1. **Hiperparâmetros do modelo** que influenciam a seleção do modelo, como a quantidade e a largura das camadas ocultas
2. **Hiperparâmetros do algoritmo** que influenciam a velocidade e a qualidade do algoritmo de aprendizado, como a taxa de aprendizado para o Stochastic Gradient Descent (SGD) e o número de vizinhos mais próximos para um classificador k Nearest Neighbors (KNN).

Neste tutorial, você usará o Keras Tuner para realizar o ajuste de hiperparâmetros para um aplicativo de classificação de imagens.

## Configuração

In [2]:
import tensorflow as tf
from tensorflow import keras

Instale e importe o Keras Tuner.

In [3]:
!pip install -q -U keras-tuner

In [4]:
import keras_tuner as kt

## Faça o download e prepare o conjunto de dados

Neste tutorial, você usará o Keras Tuner para encontrar os melhores hiperparâmetros para um modelo de aprendizado de máquina que classifica imagens de roupas do [Fashion MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).

Carregar os dados.

In [5]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 2us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [6]:
# Normalize os valores de pixel entre 0 e 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Definir o modelo

Ao criar um modelo para sitonia de hiperparâmetros, você também define o espaço de busca do hiperparâmetro, além da arquitetura do modelo. O modelo que você configura para o hipertuning é chamado de *hipermodelo*.

Você pode definir um hipermodelo por meio de duas abordagens:

* Usando uma função construtora de modelos
* Criando uma subclasse de `HyperModel` da API do Keras Tuner

Você também pode usar duas classes [HyperModel](https://keras.io/api/keras_tuner/hypermodels/) predefinidas - [HyperXception](https://keras.io/api/keras_tuner/hypermodels/hyper_xception/) e [HyperResNet](https://keras.io/api/keras_tuner/hypermodels/hyper_resnet/) para aplicativos de visão computacional.

Neste tutorial, você usa uma função construtora de modelos para definir o modelo de classificação de imagens. A função construtora de modelos retorna um modelo compilado e usa hiperparâmetros que você define em linha para hipertensionar o modelo.

In [7]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Ajuste o número de unidades na primeira camada densa
  # Escolha um valor ideal entre 32 e 512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))
  
  # Ajuste a taxa de aprendizado para o otimizador
  # Escolha um valor ideal entre 0,01, 0,001 ou 0,0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instanciar o sintonizador e executar o hipertuning

Instanciar o sintonizador para executar o hipertuning. O Keras Tuner tem quatro sintonizadores disponíveis: `RandomSearch`, `Hyperband`, `BayesianOptimization` e `Sklearn`. Neste tutorial, você usará o sintonizador [Hyperband](https://arxiv.org/pdf/1603.06560.pdf).

Para instanciar o sintonizador Hyperband, você deve especificar o hipermodelo, o `objetivo` a ser otimizado e o número máximo de épocas para treinar (`max_epochs`).

In [19]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Reloading Tuner from my_dir/intro_to_kt/tuner0.json


O algoritmo de ajuste Hyperband usa alocação adaptável de recursos e parada antecipada para convergir rapidamente em um modelo de alto desempenho.

Isso é feito usando um suporte de estilo de campeonato esportivo. O algoritmo treina um grande número de modelos por algumas épocas e leva apenas a metade dos modelos com melhor desempenho para a próxima rodada.

A hiperbanda determina o número de modelos a serem treinados calculando 1 + log<sub>`factor`</sub>(`max_epochs`) e arredondando-o para o número inteiro mais próximo.

Crie um _callback_ para interromper o treinamento antecipadamente após atingir um determinado valor para a perda de validação.

In [20]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

Executa a pesquisa de hiperparâmetros. Os argumentos para o método de pesquisa são os mesmos usados para `tf.keras.model.fit`, além do _callback_ acima.

In [21]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Obtenha os hiperparâmetros ideais
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
A pesquisa de hiperparâmetros está concluída. O número ideal de unidades na primeira camada densamente conectada
densamente conectada é {best_hps.get('units')} e a taxa de aprendizado ideal para o otimizador
é {best_hps.get('learning_rate')}.
""")


A pesquisa de hiperparâmetros está concluída. O número ideal de unidades na primeira camada densamente conectada
densamente conectada é 384 e a taxa de aprendizado ideal para o otimizador
é 0.001.



## Treinar o modelo

Encontre o número ideal de épocas para treinar o modelo com os hiperparâmetros obtidos na pesquisa.

In [22]:
# Crie o modelo com os hiperparâmetros ideais e treine-o nos dados por 50 épocas
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Melhor época: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4982 - accuracy: 0.8213 - val_loss: 0.3866 - val_accuracy: 0.8628
Epoch 2/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3713 - accuracy: 0.8646 - val_loss: 0.3782 - val_accuracy: 0.8633
Epoch 3/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.3313 - accuracy: 0.8788 - val_loss: 0.3522 - val_accuracy: 0.8739
Epoch 4/50
1500/1500 [==============================] - 3s 2ms/step - loss: 0.3089 - accuracy: 0.8864 - val_loss: 0.3556 - val_accuracy: 0.8741
Epoch 5/50
1500/1500 [==============================] - 2s 2ms/step - loss: 0.2893 - accuracy: 0.8921 - val_loss: 0.3215 - val_accuracy: 0.8867
Epoch 6/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2725 - accuracy: 0.8987 - val_loss: 0.3521 - val_accuracy: 0.8766
Epoch 7/50
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2596 - accuracy: 0.9018 - val_loss: 0.3097 - val_accuracy:

Re-instale o hipermodelo e treine-o com o número ideal de épocas acima.

In [23]:
hypermodel = tuner.hypermodel.build(best_hps)

# Treinar novamente o modelo
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.4920 - accuracy: 0.8258 - val_loss: 0.4054 - val_accuracy: 0.8569
Epoch 2/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3688 - accuracy: 0.8658 - val_loss: 0.3822 - val_accuracy: 0.8583
Epoch 3/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3323 - accuracy: 0.8785 - val_loss: 0.3449 - val_accuracy: 0.8766
Epoch 4/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.3030 - accuracy: 0.8877 - val_loss: 0.3540 - val_accuracy: 0.8741
Epoch 5/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2860 - accuracy: 0.8944 - val_loss: 0.3198 - val_accuracy: 0.8838
Epoch 6/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2704 - accuracy: 0.9000 - val_loss: 0.3559 - val_accuracy: 0.8766
Epoch 7/25
1500/1500 [==============================] - 2s 1ms/step - loss: 0.2590 - accuracy: 0.9036 - val_loss: 0.3258 - val_accuracy:

Para concluir este tutorial, avalie o hipermodelo nos dados de teste.

In [24]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 0s 606us/step - loss: 0.4142 - accuracy: 0.8892
[test loss, test accuracy]: [0.4141899049282074, 0.88919997215271]


O diretório `my_dir/intro_to_kt` contém registros detalhados e pontos de verificação(_checkpoints_) para cada tentativa (configuração de modelo) executada durante a pesquisa de hiperparâmetro. Se você executar novamente a pesquisa de hiperparâmetros, o Keras Tuner usará o estado existente desses registros para retomar a pesquisa. Para desativar esse comportamento, passe um argumento adicional `overwrite=True` ao instanciar o sintonizador.

## Resumo

Neste tutorial, você aprendeu a usar o Keras Tuner para ajustar os hiperparâmetros de um modelo. Para saber mais sobre o Keras Tuner, confira estes recursos adicionais:

* [Keras Tuner no blog do TensorFlow](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
* Site do Keras Tuner](https://keras-team.github.io/keras-tuner/)

Confira também o [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) no TensorBoard para ajustar interativamente os hiperparâmetros do modelo.